In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle
import pandas as pd
import datetime
from tqdm import tqdm
from pathlib import Path
import re
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [2]:
consolidated = pd.read_csv('../consolidated/consolidated.2020_08_23_2023_05_31.fixed_tz.mrn.csv')

/tmp/ipykernel_2255/3739912504.py:1: DtypeWarning: Columns (62) have mixed types. Specify dtype option on import or set low_memory=False.
  consolidated = pd.read_csv('../consolidated/consolidated.2020_08_23_2023_05_31.fixed_tz.mrn.csv')


In [3]:
consolidated.head()

,patient_id,roomed_time,dispo_time,waveform_start_time,waveform_end_time,visit_length_sec,data_length_sec,data_available_offset_sec,data_start_offset_sec,recommended_trim_start_sec,recommended_trim_end_sec,min_study_start,max_study_end,II_available,Pleth_available,Resp_available,studies,notes,Unnamed: 0,CSN,Age,Gender,CC,Triage_acuity,Arrival_area,Admit_area,First_room,Roomed_time,First_bed,Last_bed,ED_dispo,DC_dispo,Dispo_time,Dispo_plan_time,Decision_to_admit_time,Decision_to_CDU_time,Admit_time,Departure_time,Admit_service,ED_LOS,Hosp_LOS,Dx_ICD9,Dx_ICD10,Dx_name,Row,Arrival_time,Arrival_to_roomed,CC_CP_SOB,SpO2,RR,HR,Temp,SBP,DBP,study_matched,final_export_df_ids,final_studies,Means_of_arrival,PMD,Means_of_arrival_cat,Payor,Arrival_month,MRN
0,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,2020-10-26 13:51:59.788000-07:00,2020-10-26 15:43:59.996000-07:00,7140.0,6720.2,46649.230,46649.230,0,0,2020-10-26 00:47:30.770000-07:00,2020-10-26 16:05:32.844000-07:00,1,1,1,"STUDY-044140,STUDY-046204",NaN,12704,131295394999,18,F,CHEST PAIN,3-Urgent,SHC ED 500P ED LOBBY,SHC ED 500P BRAVO ZONE,B05,2020-10-26T13:45:00-07:00,B05,B05,Discharge,Home/Work (includes foster care),2020-10-26T15:44:00-07:00,10/26/20 15:32,NaN,NaN,NaN,2020-10-26T16:02:00-07:00,NaN,2.82,0.0,786.5,R07.9,Acute chest pain,12705.0,2020-10-26T13:13:00-07:00,32.0,1.0,100.0,16.0,72.0,37.1,120.0,71.0,True,"4720,6424","STUDY-044140,STUDY-046204",Car/Bus,"SINGH, BINDYA S",Self,BLUE CROSS MEDI-CAL,2020-10-01T00:00:00-07:00,79076386
1,131297392260,2020-12-01 12:24:00-08:00,2020-12-01 12:45:00-08:00,NaN,NaN,1260.0,0.0,81770.499,81770.499,0,0,2020-11-30 13:41:09.501000-08:00,2020-11-30 21:54:52.605000-08:00,0,0,0,STUDY-060928,no waveforms found,17953,131297392260,55,M,HYPOTENSION,2-Emergent,SHC ED 500P ED LOBBY,SHC ED 500P BRAVO ZONE,B12,2020-12-01T12:24:00-08:00,B12,B12,Admit to Inpatient,Expired,2020-12-01T12:45:00-08:00,2012-01-20 12:45,2012-01-20 12:45,NaN,2020-12-01T15:05:00-08:00,2020-12-01T17:33:00-08:00,Critical Care,5.22,9.0,"038.9, 785.52, 995.92","A41.9, R65.21",Septic shock (CMS-HCC),17954.0,2020-12-01T12:20:00-08:00,4.0,0.0,98.0,28.0,69.0,36.4,75.0,51.0,True,5057,STUDY-060928,Ambulance-American Medical Response,"LIN, STEVEN YU-TA",EMS,MEDICARE,2020-12-01T00:00:00-08:00,6448500
2,131297278364,2020-11-30 09:24:00-08:00,2020-11-30 12:56:00-08:00,2020-11-30 09:24:00.007000-08:00,2020-11-30 12:55:59.991000-08:00,12720.0,12720.0,3762.601,3762.601,0,0,2020-11-30 08:21:17.399000-08:00,2020-11-30 14:39:56.119000-08:00,1,1,1,STUDY-064313,NaN,17777,131297278364,28,M,ABDOMINAL PAIN,3-Urgent,SHC ED 300P PEDS ED LOBBY,SHC ED 500P ALPHA Z ZONE,A06,2020-11-30T09:24:00-08:00,A06,A06,Admit to Inpatient,Home/Work (includes foster care),2020-11-30T12:56:00-08:00,11/30/20 12:56,11/30/20 12:56,NaN,2020-11-30T13:22:00-08:00,2020-11-30T14:29:00-08:00,NaN,5.08,1.0,540.9,K35.80,"Acute appendicitis, unspecified acute appendic...",17778.0,2020-11-30T08:57:00-08:00,27.0,0.0,100.0,20.0,62.0,36.8,148.0,62.0,True,5857,STUDY-064313,Car/Bus,"VELADO, IVONNE L",Self,HPSM,2020-11-01T00:00:00-07:00,60170032
3,131295395670,2020-10-26 13:18:00-07:00,2020-10-26 17:35:00-07:00,2020-10-26 13:18:00.006000-07:00,2020-10-26 17:34:59.990000-07:00,15420.0,15420.0,46645.034,46645.034,587,15414,2020-10-26 00:20:34.966000-07:00,2020-10-26 18:10:21.590000-07:00,1,1,1,STUDY-048031,NaN,12706,131295395670,92,F,FALL,2-Emergent,SHC ED 500P ED LOBBY,SHC ED 500P ALPHA Y ZONE,A02,2020-10-26T13:18:00-07:00,A02,ADULT ED OVRFLW,Admit to Inpatient,Home Health Services,2020-10-26T17:35:00-07:00,10/26/20 14:52,10/26/20 14:52,NaN,2020-10-26T17:07:00-07:00,2020-10-26T18:14:00-07:00,Trauma,4.93,5.0,807.09,S22.41XA,Closed fracture of multiple ribs of right side...,12707.0,2020-10-26T13:18:00-07:00,0.0,0.0,97.0,21.0,93.0,36.7,137.0,80.0,True,8419,STUDY-048031,Ambulance,NO PCP,EMS,KAISER SENIOR,2020-10-01T00:00:00-07:00,79076485
4,131294551669,2020-10-10 12:05:00-07:00,2020-10-10 15:04:00-07:00,2020-10-10 12:05:00.016

In [5]:
consolidated.shape

(151095, 63)

In [4]:
ecg_yes = consolidated[consolidated['II_available']==1]
ecg_yes.shape

(116184, 63)

In [6]:
ecg_yes.to_csv('/home/ubuntu/ED_processed/filtered_consolidated_files/ecg_yes_116184_consolidated.csv')

In [43]:
ppg_yes = consolidated[consolidated['Pleth_available']==1]
ppg_yes.shape

(114593, 63)

In [44]:
ppg_yes.to_csv('/home/ubuntu/ED_processed/filtered_consolidated_files/ppg_yes_114593_consolidated.csv')

In [ ]:
def generate_random_time_points(start, end, num_points):
    duration = (end - start).total_seconds()
    random_points = sorted([random.uniform(0, duration) for _ in range(num_points)])
    return [start + timedelta(seconds=point) for point in random_points]

# Generate 5 random sample points
random_sample_points = generate_random_time_points(start_time, end_time, 5)
random_sample_points_formatted = [point.isoformat() for point in random_sample_points]


# ECG

# Troponin

In [7]:
trop_rows = pd.read_csv('/home/ubuntu/ED_processed/waveform/consolidated/troponin_rows.csv')
trop_rows = trop_rows[['CSN', 'Collection_time', 'Component_abnormal', 'Component_abnormal_label']]

In [8]:
trop_rows

,CSN,Collection_time,Component_abnormal,Component_abnormal_label
0,131316755104,2022-07-26T20:53:00Z,Critical High,2
1,131316755104,2022-07-27T06:36:00Z,Critical High,2
2,131316755104,2022-07-27T00:21:00Z,Critical High,2
3,131316755104,2022-07-27T13:13:00Z,Critical High,2
4,131316755104,2022-07-27T16:16:00Z,Critical High,2
...,...,...,...,...
62982,131353632713,2023-06-01T04:04:00Z,Abnormal High,1
62983,131353632713,2023-06-01T09:09:00Z,Abnormal High,1
62984,131353632815,2023-06-01T05:23:00Z,Normal,0
62985,131353634216,2023-06-01T06:55:00Z,Abnormal High,1


In [10]:
merged_trop_consolidated = pd.merge(consolidated, trop_rows, on='CSN')
merged_trop_consolidated = merged_trop_consolidated[["patient_id", "roomed_time", "dispo_time","II_available","Pleth_available" ,"waveform_start_time", "waveform_end_time", "Collection_time", "Component_abnormal_label"]]
merged_trop_consolidated

,patient_id,roomed_time,dispo_time,II_available,Pleth_available,waveform_start_time,waveform_end_time,Collection_time,Component_abnormal_label
0,131320304989,2021-11-21 15:31:00-08:00,2021-11-21 20:07:00-08:00,1,1,2021-11-21 15:35:24.015000-08:00,2021-11-21 20:06:59.999000-08:00,2022-04-11T04:00:00Z,1
1,131320304989,2021-11-21 15:31:00-08:00,2021-11-21 20:07:00-08:00,1,1,2021-11-21 15:35:24.015000-08:00,2021-11-21 20:06:59.999000-08:00,2022-04-14T20:29:00Z,0
2,131321239988,2021-12-07 20:06:00-08:00,2021-12-07 20:31:00-08:00,1,1,2021-12-07 20:06:00.007000-08:00,2021-12-07 20:30:59.991000-08:00,2022-03-03T18:20:00Z,1
3,131321239988,2021-12-07 20:06:00-08:00,2021-12-07 20:31:00-08:00,1,1,2021-12-07 20:06:00.007000-08:00,2021-12-07 20:30:59.991000-08:00,2022-03-04T16:19:00Z,2
4,131321239988,2021-12-07 20:06:00-08:00,2021-12-07 20:31:00-08:00,1,1,2021-12-07 20:06:00.007000-08:00,2021-12-07 20:30:59.991000-08:00,2022-03-04T21:00:00Z,2
...,...,...,...,...,...,...,...,...,...
53130,131346525439,2023-02-04 09:17:00-08:00,2023-02-04 13:41:00-08:00,1,1,2023-02-04 10:48:53.412000-08:00,2023-02-04 13:40:59.988000-08:00,2023-02-04T18:33:00Z,1
53131,131346525439,2023-02-04 09:17:00-08:00,2023-02-04 13:41:00-08:00,1,1,2023-02-04 10:48:53.412000-08:00,2023-02-04 13:40:59.988000-08:00,2023-02-04T20:34:00Z,1
53132,131349974033,2023-04-02 01:09:00-07:00,2023-04-02 04:43:00-07:00,1,1,2023-04-02 01:43:10.545000-07:00,2023-04-02 04:42:59.985000-07:00,2023-04-02T08:21:00Z,0
53133,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-02 21:39:26.848000-07:00,2023-04-02 23:02:59.984000-07:00,2023-04-03T04:37:00Z,2


In [14]:
merged_trop_consolidated['Collection_time'] = pd.to_datetime(merged_trop_consolidated['Collection_time'], errors='coerce', utc=True)
merged_trop_consolidated['waveform_end_time'] = pd.to_datetime(merged_trop_consolidated['waveform_end_time'], errors='coerce', utc=True)
merged_trop_consolidated['waveform_start_time'] = pd.to_datetime(merged_trop_consolidated['waveform_start_time'], errors='coerce', utc=True)

In [15]:
valid_trop_waveform = merged_trop_consolidated[merged_trop_consolidated['waveform_end_time']>=merged_trop_consolidated['Collection_time']]
valid_trop_waveform = valid_trop_waveform[valid_trop_waveform['waveform_start_time']<valid_trop_waveform['Collection_time']]
valid_trop_waveform

,patient_id,roomed_time,dispo_time,II_available,Pleth_available,waveform_start_time,waveform_end_time,Collection_time,Component_abnormal_label
41,131327307435,2022-03-17 20:59:00-07:00,2022-03-17 23:47:00-07:00,1,1,2022-03-18 04:30:34.841000+00:00,2022-03-18 06:46:59.993000+00:00,2022-03-18 06:05:00+00:00,1
42,131327307676,2022-03-17 19:57:00-07:00,2022-03-17 23:45:00-07:00,1,1,2022-03-18 02:57:00.011000+00:00,2022-03-18 06:44:59.995000+00:00,2022-03-18 03:27:00+00:00,0
51,131327308349,2022-03-17 21:28:00-07:00,2022-03-17 23:32:00-07:00,1,1,2022-03-18 04:33:51.760000+00:00,2022-03-18 06:31:59.984000+00:00,2022-03-18 04:41:00+00:00,0
53,131327308606,2022-03-17 21:56:00-07:00,2022-03-18 00:43:00-07:00,1,1,2022-03-18 05:04:52.064000+00:00,2022-03-18 07:42:59.984000+00:00,2022-03-18 06:26:00+00:00,1
54,131327308831,2022-03-17 21:15:00-07:00,2022-03-18 00:15:00-07:00,1,1,2022-03-18 04:15:03.270000+00:00,2022-03-18 07:14:59.990000+00:00,2022-03-18 04:38:00+00:00,1
...,...,...,...,...,...,...,...,...,...
53124,131342590197,2022-12-01 18:19:00-08:00,2022-12-02 00:25:00-08:00,1,1,2022-12-02 02:19:32.642000+00:00,2022-12-02 08:24:59.986000+00:00,2022-12-02 03:17:00+00:00,1
53126,131342621813,2022-12-02 10:33:00-08:00,2022-12-02 14:34:00-08:00,1,1,2022-12-02 18:44:39.665000+00:00,2022-12-02 22:33:59.985000+00:00,2022-12-02 20:47:00+00:00,0
53127,131342684157,2022-12-03 13:09:00-08:00,2022-12-03 16:35:00-08:00,1,1,2022-12-03 21:17:40.214000+00:00,2022-12-04 00:34:59.990000+00:00,2022-12-03 21:23:00+00:00,0
53128,131346404473,2023-02-02 14:52:00-08:00,2023-02-02 16:13:00-08:00,1,1,2023-02-02 22:58:57.037000+00:00,2023-02-03 00:12:59.997000+00:00,2023-02-02 23:07:00+00:00,0


In [22]:
valid_trop_waveform.to_csv('/home/ubuntu/ED_processed/filtered_consolidated_files/ecg_yes_troponin_25161_consolidated.csv')

# Potassium

In [17]:
k_rows = pd.read_csv('/home/ubuntu/ED_processed/waveform/consolidated/potassium_rows.csv')
k_rows = k_rows[['CSN', 'Collection_time', 'Component_abnormal', 'Component_abnormal_label']]

In [18]:
merged_k_consolidated = pd.merge(consolidated, k_rows, on='CSN')
merged_k_consolidated = merged_k_consolidated[["patient_id", "roomed_time", "dispo_time","II_available","Pleth_available" ,"waveform_start_time", "waveform_end_time", "Collection_time", "Component_abnormal_label"]]
merged_k_consolidated

,patient_id,roomed_time,dispo_time,II_available,Pleth_available,waveform_start_time,waveform_end_time,Collection_time,Component_abnormal_label
0,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 13:51:59.788000-07:00,2020-10-26 15:43:59.996000-07:00,2020-10-26T21:26:00Z,0
1,131297278364,2020-11-30 09:24:00-08:00,2020-11-30 12:56:00-08:00,1,1,2020-11-30 09:24:00.007000-08:00,2020-11-30 12:55:59.991000-08:00,2020-11-30T18:05:00Z,0
2,131297278364,2020-11-30 09:24:00-08:00,2020-11-30 12:56:00-08:00,1,1,2020-11-30 09:24:00.007000-08:00,2020-11-30 12:55:59.991000-08:00,2020-12-01T15:06:00Z,0
3,131295395670,2020-10-26 13:18:00-07:00,2020-10-26 17:35:00-07:00,1,1,2020-10-26 13:18:00.006000-07:00,2020-10-26 17:34:59.990000-07:00,2020-10-26T20:32:00Z,0
4,131295395670,2020-10-26 13:18:00-07:00,2020-10-26 17:35:00-07:00,1,1,2020-10-26 13:18:00.006000-07:00,2020-10-26 17:34:59.990000-07:00,2020-10-27T13:56:00Z,0
...,...,...,...,...,...,...,...,...,...
487389,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-02 21:39:26.848000-07:00,2023-04-02 23:02:59.984000-07:00,2023-04-03T04:37:00Z,0
487390,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-02 21:39:26.848000-07:00,2023-04-02 23:02:59.984000-07:00,2023-04-03T17:21:00Z,0
487391,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-02 21:39:26.848000-07:00,2023-04-02 23:02:59.984000-07:00,2023-04-04T13:29:00Z,0
487392,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-02 21:39:26.848000-07:00,2023-04-02 23:02:59.984000-07:00,2023-04-05T13:33:00Z,3


In [19]:
merged_k_consolidated['Collection_time'] = pd.to_datetime(merged_k_consolidated['Collection_time'], errors='coerce', utc=True)
merged_k_consolidated['waveform_end_time'] = pd.to_datetime(merged_k_consolidated['waveform_end_time'], errors='coerce', utc=True)
merged_k_consolidated['waveform_start_time'] = pd.to_datetime(merged_k_consolidated['waveform_start_time'], errors='coerce', utc=True)

In [20]:
valid_k_waveform = merged_k_consolidated[merged_k_consolidated['waveform_end_time']>=merged_k_consolidated['Collection_time']]
valid_k_waveform = valid_k_waveform[valid_k_waveform['waveform_start_time']<valid_k_waveform['Collection_time']]
valid_k_waveform

,patient_id,roomed_time,dispo_time,II_available,Pleth_available,waveform_start_time,waveform_end_time,Collection_time,Component_abnormal_label
0,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 21:26:00+00:00,0
1,131297278364,2020-11-30 09:24:00-08:00,2020-11-30 12:56:00-08:00,1,1,2020-11-30 17:24:00.007000+00:00,2020-11-30 20:55:59.991000+00:00,2020-11-30 18:05:00+00:00,0
3,131295395670,2020-10-26 13:18:00-07:00,2020-10-26 17:35:00-07:00,1,1,2020-10-26 20:18:00.006000+00:00,2020-10-27 00:34:59.990000+00:00,2020-10-26 20:32:00+00:00,0
5,131292831365,2020-09-16 17:40:00-07:00,2020-09-16 22:38:00-07:00,1,1,2020-09-17 00:40:00.015000+00:00,2020-09-17 05:37:59.999000+00:00,2020-09-17 00:57:00+00:00,0
12,131294551643,2020-10-10 12:02:00-07:00,2020-10-10 13:36:00-07:00,1,1,2020-10-10 19:05:33.707000+00:00,2020-10-10 20:35:59.995000+00:00,2020-10-10 19:34:00+00:00,0
...,...,...,...,...,...,...,...,...,...
487346,131346383433,2023-02-02 09:45:00-08:00,2023-02-02 13:38:00-08:00,1,1,2023-02-02 17:55:01.758000+00:00,2023-02-02 21:37:59.998000+00:00,2023-02-02 18:24:00+00:00,0
487370,131349976622,2023-04-02 09:56:00-07:00,2023-04-02 11:16:00-07:00,1,1,2023-04-02 17:02:22.053000+00:00,2023-04-02 18:15:59.989000+00:00,2023-04-02 17:19:00+00:00,0
487371,131349976622,2023-04-02 09:56:00-07:00,2023-04-02 11:16:00-07:00,1,1,2023-04-02 17:02:22.053000+00:00,2023-04-02 18:15:59.989000+00:00,2023-04-02 17:18:00+00:00,0
487372,131349978582,2023-04-02 12:21:00-07:00,2023-04-02 14:26:00-07:00,1,1,2023-04-02 19:29:30.417000+00:00,2023-04-02 21:25:59.985000+00:00,2023-04-02 20:12:00+00:00,0


In [21]:
valid_trop_waveform.to_csv('/home/ubuntu/ED_processed/filtered_consolidated_files/ecg_yes_potassium_69937_consolidated.csv')

# Calcium

In [23]:
ca_rows = pd.read_csv('/home/ubuntu/ED_processed/waveform/consolidated/calcium_rows.csv')
ca_rows = ca_rows[['CSN', 'Collection_time', 'Component_abnormal', 'Component_abnormal_label']]
merged_ca_consolidated = pd.merge(consolidated, ca_rows, on='CSN')
merged_ca_consolidated = merged_ca_consolidated[["patient_id", "roomed_time", "dispo_time","II_available","Pleth_available" ,"waveform_start_time", "waveform_end_time", "Collection_time", "Component_abnormal_label"]]
merged_ca_consolidated

/tmp/ipykernel_2255/3137243107.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  ca_rows = pd.read_csv('/home/ubuntu/ED_processed/waveform/consolidated/calcium_rows.csv')


,patient_id,roomed_time,dispo_time,II_available,Pleth_available,waveform_start_time,waveform_end_time,Collection_time,Component_abnormal_label
0,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 13:51:59.788000-07:00,2020-10-26 15:43:59.996000-07:00,2020-10-26T21:26:00Z,0
1,131297278364,2020-11-30 09:24:00-08:00,2020-11-30 12:56:00-08:00,1,1,2020-11-30 09:24:00.007000-08:00,2020-11-30 12:55:59.991000-08:00,2020-11-30T18:05:00Z,0
2,131297278364,2020-11-30 09:24:00-08:00,2020-11-30 12:56:00-08:00,1,1,2020-11-30 09:24:00.007000-08:00,2020-11-30 12:55:59.991000-08:00,2020-12-01T15:06:00Z,0
3,131295395670,2020-10-26 13:18:00-07:00,2020-10-26 17:35:00-07:00,1,1,2020-10-26 13:18:00.006000-07:00,2020-10-26 17:34:59.990000-07:00,2020-10-26T20:32:00Z,0
4,131295395670,2020-10-26 13:18:00-07:00,2020-10-26 17:35:00-07:00,1,1,2020-10-26 13:18:00.006000-07:00,2020-10-26 17:34:59.990000-07:00,2020-10-27T13:56:00Z,0
...,...,...,...,...,...,...,...,...,...
506783,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-02 21:39:26.848000-07:00,2023-04-02 23:02:59.984000-07:00,2023-04-03T04:37:00Z,0
506784,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-02 21:39:26.848000-07:00,2023-04-02 23:02:59.984000-07:00,2023-04-03T17:21:00Z,5
506785,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-02 21:39:26.848000-07:00,2023-04-02 23:02:59.984000-07:00,2023-04-04T13:29:00Z,5
506786,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-02 21:39:26.848000-07:00,2023-04-02 23:02:59.984000-07:00,2023-04-05T13:33:00Z,0


In [24]:
merged_ca_consolidated['Collection_time'] = pd.to_datetime(merged_ca_consolidated['Collection_time'], errors='coerce', utc=True)
merged_ca_consolidated['waveform_end_time'] = pd.to_datetime(merged_ca_consolidated['waveform_end_time'], errors='coerce', utc=True)
merged_ca_consolidated['waveform_start_time'] = pd.to_datetime(merged_ca_consolidated['waveform_start_time'], errors='coerce', utc=True)
valid_ca_waveform = merged_ca_consolidated[merged_ca_consolidated['waveform_end_time']>=merged_ca_consolidated['Collection_time']]
valid_ca_waveform = valid_ca_waveform[valid_ca_waveform['waveform_start_time']<valid_ca_waveform['Collection_time']]
valid_ca_waveform

,patient_id,roomed_time,dispo_time,II_available,Pleth_available,waveform_start_time,waveform_end_time,Collection_time,Component_abnormal_label
0,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 21:26:00+00:00,0
1,131297278364,2020-11-30 09:24:00-08:00,2020-11-30 12:56:00-08:00,1,1,2020-11-30 17:24:00.007000+00:00,2020-11-30 20:55:59.991000+00:00,2020-11-30 18:05:00+00:00,0
3,131295395670,2020-10-26 13:18:00-07:00,2020-10-26 17:35:00-07:00,1,1,2020-10-26 20:18:00.006000+00:00,2020-10-27 00:34:59.990000+00:00,2020-10-26 20:32:00+00:00,0
5,131292831365,2020-09-16 17:40:00-07:00,2020-09-16 22:38:00-07:00,1,1,2020-09-17 00:40:00.015000+00:00,2020-09-17 05:37:59.999000+00:00,2020-09-17 00:57:00+00:00,4
12,131294551643,2020-10-10 12:02:00-07:00,2020-10-10 13:36:00-07:00,1,1,2020-10-10 19:05:33.707000+00:00,2020-10-10 20:35:59.995000+00:00,2020-10-10 19:34:00+00:00,0
...,...,...,...,...,...,...,...,...,...
506731,131342621813,2022-12-02 10:33:00-08:00,2022-12-02 14:34:00-08:00,1,1,2022-12-02 18:44:39.665000+00:00,2022-12-02 22:33:59.985000+00:00,2022-12-02 18:53:00+00:00,0
506735,131346383433,2023-02-02 09:45:00-08:00,2023-02-02 13:38:00-08:00,1,1,2023-02-02 17:55:01.758000+00:00,2023-02-02 21:37:59.998000+00:00,2023-02-02 18:24:00+00:00,0
506765,131349976622,2023-04-02 09:56:00-07:00,2023-04-02 11:16:00-07:00,1,1,2023-04-02 17:02:22.053000+00:00,2023-04-02 18:15:59.989000+00:00,2023-04-02 17:19:00+00:00,0
506766,131349978582,2023-04-02 12:21:00-07:00,2023-04-02 14:26:00-07:00,1,1,2023-04-02 19:29:30.417000+00:00,2023-04-02 21:25:59.985000+00:00,2023-04-02 20:12:00+00:00,0


In [25]:
valid_trop_waveform.to_csv('/home/ubuntu/ED_processed/filtered_consolidated_files/ecg_yes_calcium_67376_consolidated.csv')

# PPG

In [45]:
consolidated = ppg_yes

# Lactate

In [46]:
rows = pd.read_csv('/home/ubuntu/ED_processed/waveform/consolidated/lactate_rows.csv')
rows = rows[['CSN', 'Collection_time', 'Component_name', 'Component_abnormal', 'Component_abnormal_label']]
merged_consolidated = pd.merge(consolidated, rows, on='CSN')
merged_consolidated = merged_consolidated[["patient_id", "roomed_time", "dispo_time","II_available","Pleth_available" ,"waveform_start_time", "waveform_end_time", "Collection_time", "Component_abnormal_label", "Component_name"]]
merged_consolidated['Collection_time'] = pd.to_datetime(merged_consolidated['Collection_time'], errors='coerce', utc=True)
merged_consolidated['waveform_end_time'] = pd.to_datetime(merged_consolidated['waveform_end_time'], errors='coerce', utc=True)
merged_consolidated['waveform_start_time'] = pd.to_datetime(merged_consolidated['waveform_start_time'], errors='coerce', utc=True)
valid_waveform = merged_consolidated[merged_consolidated['waveform_end_time']>=merged_consolidated['Collection_time']]
valid_waveform = valid_waveform[valid_waveform['waveform_start_time']<valid_waveform['Collection_time']]
valid_waveform

,patient_id,roomed_time,dispo_time,II_available,Pleth_available,waveform_start_time,waveform_end_time,Collection_time,Component_abnormal_label,Component_name
0,131295395670,2020-10-26 13:18:00-07:00,2020-10-26 17:35:00-07:00,1,1,2020-10-26 20:18:00.006000+00:00,2020-10-27 00:34:59.990000+00:00,2020-10-26 20:44:00+00:00,2,"POC:LACTATE, ISTAT"
1,131292831365,2020-09-16 17:40:00-07:00,2020-09-16 22:38:00-07:00,1,1,2020-09-17 00:40:00.015000+00:00,2020-09-17 05:37:59.999000+00:00,2020-09-17 01:01:00+00:00,0,"POC:LACTATE, ISTAT"
3,131295400006,2020-10-26 15:02:00-07:00,2020-10-26 21:04:00-07:00,1,1,2020-10-26 22:18:09.557000+00:00,2020-10-27 04:03:59.989000+00:00,2020-10-26 22:47:00+00:00,0,"POC:LACTATE, ISTAT"
4,131294551364,2020-10-10 11:37:00-07:00,2020-10-10 16:32:00-07:00,1,1,2020-10-10 18:37:00.005000+00:00,2020-10-10 23:31:59.989000+00:00,2020-10-10 18:41:00+00:00,2,"POC:LACTATE, ISTAT"
7,131297391127,2020-12-01 12:12:00-08:00,2020-12-01 15:14:00-08:00,1,1,2020-12-01 20:17:03.629000+00:00,2020-12-01 23:13:59.997000+00:00,2020-12-01 20:37:00+00:00,0,"POC:LACTATE, ISTAT"
...,...,...,...,...,...,...,...,...,...,...
98958,131342590197,2022-12-01 18:19:00-08:00,2022-12-02 00:25:00-08:00,1,1,2022-12-02 02:19:32.642000+00:00,2022-12-02 08:24:59.986000+00:00,2022-12-02 06:19:00+00:00,0,"POC:LACTATE, ISTAT"
98961,131342592390,2022-12-01 20:16:00-08:00,2022-12-02 00:05:00-08:00,1,1,2022-12-02 04:41:46.582000+00:00,2022-12-02 08:04:59.990000+00:00,2022-12-02 04:53:00+00:00,0,"POC:LACTATE, ISTAT"
98962,131346383433,2023-02-02 09:45:00-08:00,2023-02-02 13:38:00-08:00,1,1,2023-02-02 17:55:01.758000+00:00,2023-02-02 21:37:59.998000+00:00,2023-02-02 18:45:00+00:00,0,"POC:LACTATE, ISTAT"
98964,131349981274,2023-04-02 16:04:00-07:00,2023-04-02 20:28:00-07:00,1,1,2023-04-02 23:16:11.732000+00:00,2023-04-03 03:27:59.988000+00:00,2023-04-02 23:32:00+00:00,2,"POC:LACTATE, ISTAT"


In [47]:
valid_waveform.to_csv('/home/ubuntu/ED_processed/filtered_consolidated_files/ecg_yes_lactate_31783_consolidated.csv')

# Respirotary Rate (RR)

In [48]:
rows = pd.read_csv('/home/ubuntu/ED_processed/waveform/consolidated/RR_rows.csv')
rows = rows[['CSN', 'Time', 'Value', 'Measure']]
merged_consolidated = pd.merge(consolidated, rows, on='CSN')
merged_consolidated = merged_consolidated[["patient_id", "roomed_time", "dispo_time","II_available","Pleth_available" ,"waveform_start_time", "waveform_end_time", "Time", "Value", "Measure"]]
merged_consolidated['Time'] = pd.to_datetime(merged_consolidated['Time'], errors='coerce', utc=True)
merged_consolidated['waveform_end_time'] = pd.to_datetime(merged_consolidated['waveform_end_time'], errors='coerce', utc=True)
merged_consolidated['waveform_start_time'] = pd.to_datetime(merged_consolidated['waveform_start_time'], errors='coerce', utc=True)
valid_waveform = merged_consolidated[merged_consolidated['waveform_end_time']>=merged_consolidated['Time']]
valid_waveform = valid_waveform[valid_waveform['waveform_start_time']<valid_waveform['Time']]
valid_waveform

,patient_id,roomed_time,dispo_time,II_available,Pleth_available,waveform_start_time,waveform_end_time,Time,Value,Measure
0,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 20:53:00+00:00,16.000000,RR
1,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 21:00:00+00:00,16.000000,RR
2,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 22:00:00+00:00,16.000000,RR
5,131295395670,2020-10-26 13:18:00-07:00,2020-10-26 17:35:00-07:00,1,1,2020-10-26 20:18:00.006000+00:00,2020-10-27 00:34:59.990000+00:00,2020-10-26 20:25:00+00:00,21.000000,RR
6,131295395670,2020-10-26 13:18:00-07:00,2020-10-26 17:35:00-07:00,1,1,2020-10-26 20:18:00.006000+00:00,2020-10-27 00:34:59.990000+00:00,2020-10-26 20:28:37+00:00,22.000000,RR
...,...,...,...,...,...,...,...,...,...,...
9803904,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 05:58:00+00:00,17.847458,RR
9803905,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 05:59:00+00:00,17.758621,RR
9803906,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 06:00:00+00:00,18.186441,RR
9803907,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 06:01:00+00:00,20.779661,RR


In [49]:
valid_waveform.to_csv('/home/ubuntu/ED_processed/filtered_consolidated_files/ecg_yes_rr_9386701_consolidated.csv')

# Blood Pressure (SBP)

In [50]:
rows = pd.read_csv('/home/ubuntu/ED_processed/waveform/consolidated/SBP_rows.csv')
rows = rows[['CSN', 'Time', 'Value', 'Measure']]
merged_consolidated = pd.merge(consolidated, rows, on='CSN')
merged_consolidated = merged_consolidated[["patient_id", "roomed_time", "dispo_time","II_available","Pleth_available" ,"waveform_start_time", "waveform_end_time", "Time", "Value", "Measure"]]
merged_consolidated['Time'] = pd.to_datetime(merged_consolidated['Time'], errors='coerce', utc=True)
merged_consolidated['waveform_end_time'] = pd.to_datetime(merged_consolidated['waveform_end_time'], errors='coerce', utc=True)
merged_consolidated['waveform_start_time'] = pd.to_datetime(merged_consolidated['waveform_start_time'], errors='coerce', utc=True)
valid_waveform = merged_consolidated[merged_consolidated['waveform_end_time']>=merged_consolidated['Time']]
valid_waveform = valid_waveform[valid_waveform['waveform_start_time']<valid_waveform['Time']]
valid_waveform

,patient_id,roomed_time,dispo_time,II_available,Pleth_available,waveform_start_time,waveform_end_time,Time,Value,Measure
0,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 20:53:00+00:00,120.0,SBP
1,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 21:00:00+00:00,118.0,SBP
2,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 21:01:00+00:00,118.0,SBP
3,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 21:32:00+00:00,126.0,SBP
4,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 21:46:00+00:00,112.0,SBP
...,...,...,...,...,...,...,...,...,...,...
1360202,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 04:42:00+00:00,117.0,SBP
1360203,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 05:45:00+00:00,112.0,SBP
1360204,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 05:46:00+00:00,112.0,SBP
1360205,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 06:00:00+00:00,109.0,SBP


In [51]:
valid_waveform.to_csv('/home/ubuntu/ED_processed/filtered_consolidated_files/ecg_yes_SBP_1011876_consolidated.csv')

# Blood Pressure (DBP)

In [52]:
rows = pd.read_csv('/home/ubuntu/ED_processed/waveform/consolidated/DBP_rows.csv')
rows = rows[['CSN', 'Time', 'Value', 'Measure']]
merged_consolidated = pd.merge(consolidated, rows, on='CSN')
merged_consolidated = merged_consolidated[["patient_id", "roomed_time", "dispo_time","II_available","Pleth_available" ,"waveform_start_time", "waveform_end_time", "Time", "Value", "Measure"]]
merged_consolidated['Time'] = pd.to_datetime(merged_consolidated['Time'], errors='coerce', utc=True)
merged_consolidated['waveform_end_time'] = pd.to_datetime(merged_consolidated['waveform_end_time'], errors='coerce', utc=True)
merged_consolidated['waveform_start_time'] = pd.to_datetime(merged_consolidated['waveform_start_time'], errors='coerce', utc=True)
valid_waveform = merged_consolidated[merged_consolidated['waveform_end_time']>=merged_consolidated['Time']]
valid_waveform = valid_waveform[valid_waveform['waveform_start_time']<valid_waveform['Time']]
valid_waveform

,patient_id,roomed_time,dispo_time,II_available,Pleth_available,waveform_start_time,waveform_end_time,Time,Value,Measure
0,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 20:53:00+00:00,71.0,DBP
1,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 21:00:00+00:00,68.0,DBP
2,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 21:01:00+00:00,68.0,DBP
3,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 21:32:00+00:00,76.0,DBP
4,131295394999,2020-10-26 13:45:00-07:00,2020-10-26 15:44:00-07:00,1,1,2020-10-26 20:51:59.788000+00:00,2020-10-26 22:43:59.996000+00:00,2020-10-26 21:46:00+00:00,79.0,DBP
...,...,...,...,...,...,...,...,...,...,...
1360202,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 04:42:00+00:00,73.0,DBP
1360203,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 05:45:00+00:00,74.0,DBP
1360204,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 05:46:00+00:00,64.0,DBP
1360205,131349984482,2023-04-02 21:29:00-07:00,2023-04-02 23:03:00-07:00,1,1,2023-04-03 04:39:26.848000+00:00,2023-04-03 06:02:59.984000+00:00,2023-04-03 06:00:00+00:00,68.0,DBP


In [53]:
valid_waveform.to_csv('/home/ubuntu/ED_processed/filtered_consolidated_files/ecg_yes_DBP_1011876_consolidated.csv')